<a href="https://colab.research.google.com/github/DarleneJD/ACOPF/blob/main/13barrasOPF_Reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [108]:
import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [109]:


import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://matematica.unipv.it/gualandi/solvers/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt
        except:
            pass

In [110]:
# if not (shutil.which("glpk") or os.path.isfile("glpk")):
#     if "google.colab" in sys.modules:
#         !apt-get install -y -qq glpk-utils
#     else:
#         try:
#             !conda install -c conda-forge glpk
#         except:
#             pass

In [111]:
!git clone https://github.com/DarleneJD/ACOPF.git

fatal: destination path 'ACOPF' already exists and is not an empty directory.


In [40]:
import math
from pathlib import Path
import pandas as pd
import pyomo.environ as pyo
from collections import defaultdict
# Solver (HiGHS)
from pyomo.environ import *
import pandas as pd
import numpy as np


In [41]:
base_path = "/content/ACOPF"


# MT

In [ ]:
branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

df_branches = pd.read_excel(branches_path, sheet_name="MT")
df_trafos   = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses    = pd.read_excel(buses_path,    sheet_name="MT")


In [ ]:
def normalize_phases(value):
    """
    Converte entradas como:
    1
    1.0
    '1,2,3'
    '1.0,2.0,3.0'
    '1.2.3'
    em lista de inteiros [1], [1,2,3]
    """
    if pd.isna(value):
        return []

    s = str(value).strip()

    # unifica separadores
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1, 2, 3]:
                phases.append(ph)
        except ValueError:
            pass

    return sorted(set(phases))

# Branches (MT)
df_branches["phase_list"] = df_branches["phase"].apply(normalize_phases)

# Trafos
df_trafos["phase_list"] = df_trafos["phases"].apply(normalize_phases)


In [ ]:
# Remover SourceBus
df_buses = df_buses[~df_buses["name"].astype(str).str.contains("Source", case=False)]

# Mapeamento name -> N (id numérico)
bus_id_map = dict(zip(df_buses["name"], df_buses["N"]))

# Lista final de barras MT
buses_mt = df_buses["N"].astype(int).unique().tolist()


In [ ]:
# Converter barras l,k para identificador numérico
df_branches["from_bus"] = df_branches["l"].map(bus_id_map)
df_branches["to_bus"]   = df_branches["k"].map(bus_id_map)

# Remover ramos mal mapeados
df_branches = df_branches.dropna(subset=["from_bus", "to_bus"])

df_branches["from_bus"] = df_branches["from_bus"].astype(int)
df_branches["to_bus"]   = df_branches["to_bus"].astype(int)
df_branches["m"]        = df_branches["m"].astype(int)

# Conjunto de ramos MT
branches_mt = df_branches["m"].unique().tolist()


In [ ]:
PHASES = [1, 2, 3]

branch_phase = []

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # garante inteiro

    raw = row["phase"]

    if pd.isna(raw):
        continue

    s = str(raw).strip()

    # Normalização:
    # troca qualquer separador por vírgula
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]

    for p in parts:
        # extrai apenas números válidos
        try:
            ph = int(float(p))
        except ValueError:
            continue

        if ph in PHASES:
            branch_phase.append((m, ph))

branch_phase = sorted(set(branch_phase))


In [ ]:
R = {}
X = {}
Imax = {}
from_bus = {}
to_bus = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))  # robusto se vier 1.0

    from_bus[m] = int(row["from_bus"])
    to_bus[m]   = int(row["to_bus"])
    Imax[m]     = float(row["Imax"])

    # USA A COLUNA NORMALIZADA
    for ph in row["phase_list"]:
        R[(m, ph)] = float(row["R"])
        X[(m, ph)] = float(row["X"])



Length = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    Length[m] = float(row["Length_m"])

Vbase = {}

for _, row in df_branches.iterrows():
    m = int(float(row["m"]))
    fb = from_bus[m]
    Vbase[m] = float(df_buses.loc[df_buses["N"] == fb, "v_nom_kv"].iloc[0])
Smax = {}

for m in branches_mt:
    Smax[m] = Vbase[m] * Imax[m]  # kVA por fase


In [ ]:
# Padronizar mv_bus
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].map(bus_id_map)
df_trafos = df_trafos.dropna(subset=["mv_bus_num"])
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)

# Placeholder de potência vinda da BT
P_TR = {}
Q_TR = {}

for _, row in df_trafos.iterrows():
    mvb = int(row["mv_bus_num"])

    # USA DIRETAMENTE A LISTA NORMALIZADA
    for ph in row["phase_list"]:
        P_TR[(mvb, ph)] = 0.0
        Q_TR[(mvb, ph)] = 0.0


In [ ]:
trafo_mv_bus = {
    row["trafo_id"]: int(row["mv_bus_num"])
    for _, row in df_trafos.iterrows()
}


alpha_tr = {}
beta_tr  = {}

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    r_per = float(row["r_per"]) / 100.0
    xhl   = float(row["XHL"]) / 100.0

    for ph in row["phase_list"]:
        alpha_tr[(tid, ph)] = r_per
        beta_tr[(tid, ph)]  = xhl


In [ ]:
model = ConcreteModel()

# Conjuntos
model.BUS = Set(initialize=buses_mt)
T = list(range(0, 2881))  # 0 ... 2880

model.T = Set(initialize=T, ordered=True)


bus_phase = set()

for (br, ph) in branch_phase:
    bus_phase.add((from_bus[br], ph))
    bus_phase.add((to_bus[br], ph))

model.BUS_PH = Set(dimen=2, initialize=bus_phase)


model.PH  = Set(initialize=PHASES)
model.BR  = Set(initialize=branches_mt)
model.BR_PH = Set(initialize=branch_phase, dimen=2)

trafo_phase = []

for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    for ph in row["phase_list"]:
        trafo_phase.append((tid, ph))

model.TR_PH = Set(dimen=2, initialize=trafo_phase)


# Parâmetros
model.R = Param(model.BR_PH, initialize=R)
model.X = Param(model.BR_PH, initialize=X)

model.from_bus = Param(model.BR, initialize=from_bus)
model.to_bus   = Param(model.BR, initialize=to_bus)



# Variáveis
model.P = Var(model.BR_PH)
model.Q = Var(model.BR_PH)
model.v = Var(model.BUS, model.PH, bounds=(0.95**2, 1.05**2))

model.Ppos = Var(model.BR_PH, domain=NonNegativeReals)
model.Pneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Qpos = Var(model.BR_PH, domain=NonNegativeReals)
model.Qneg = Var(model.BR_PH, domain=NonNegativeReals)
model.Ploss = Var(model.BR_PH, domain=NonNegativeReals)




In [ ]:
# Potência aparente "equivalente" (L1) por trafo e fase
model.Str_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Str_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Ploss_tr = Var(model.TR_PH, domain=NonNegativeReals)
model.Qloss_tr = Var(model.TR_PH, domain=NonNegativeReals)

model.Ptr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Ptr_neg = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_pos = Var(model.TR_PH, domain=NonNegativeReals)
model.Qtr_neg = Var(model.TR_PH, domain=NonNegativeReals)




In [ ]:
def Ptr_def(m, tid, ph):
    return m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]

def Qtr_def(m, tid, ph):
    return m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]


In [ ]:
def voltage_drop_rule(m, br, ph):
    i = m.from_bus[br]
    j = m.to_bus[br]
    return m.v[j,ph] == m.v[i,ph] - 2*(
        (m.R[br,ph]*Length[br]) * m.P[br,ph] +
        (m.X[br,ph]*Length[br]) * m.Q[br,ph]
    )



In [ ]:
def Ploss_rule(m, br, ph):
    return m.Ploss[br,ph] >= (
        m.R[br,ph] *
        (m.Ppos[br,ph] + m.Pneg[br,ph]
         + m.Qpos[br,ph] + m.Qneg[br,ph])
    )

model.PlossDef = Constraint(model.BR_PH, rule=Ploss_rule)


In [ ]:
def active_balance_rule(m, bus, ph):

    inflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.P[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph] + m.Ploss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

def reactive_balance_rule(m, bus, ph):

    inflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.to_bus[br] == bus
    )

    outflow = sum(
        m.Q[br, ph]
        for (br,p) in m.BR_PH
        if p == ph and m.from_bus[br] == bus
    )

    trafo_power = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph] + m.Qloss_tr[tid,ph]
        for (tid,p) in m.TR_PH
        if p == ph and trafo_mv_bus[tid] == bus
    )

    return inflow - outflow == trafo_power

model.PBalance = Constraint(model.BUS_PH, rule=active_balance_rule)
model.QBalance = Constraint(model.BUS_PH, rule=reactive_balance_rule)


In [ ]:
Vmin = 0.95  # pu

def current_limit_rule(m, br, ph):
    return (
        m.Ppos[br,ph] + m.Pneg[br,ph]
        + m.Qpos[br,ph] + m.Qneg[br,ph]
        <= Smax[br] * Vmin
    )

model.CurrentLimit = Constraint(model.BR_PH, rule=current_limit_rule)



In [ ]:
slack_bus = int(df_buses[df_buses["tb"] == 3]["N"].iloc[0])

for ph in PHASES:
    model.v[slack_bus, ph].fix(1.0)


In [ ]:
def Ploss_tr_rule(m, tid, ph):
    return m.Ploss_tr[tid,ph] >= alpha_tr[(tid,ph)] * (
        m.Ptr_pos[tid,ph] + m.Ptr_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.PlossTrDef = Constraint(model.TR_PH, rule=Ploss_tr_rule)


def Qloss_tr_rule(m, tid,ph):
    return m.Qloss_tr[tid,ph] >= beta_tr[(tid,ph)] * (
        m.Str_pos[tid,ph] + m.Str_neg[tid,ph]
        + m.Qtr_pos[tid,ph] + m.Qtr_neg[tid,ph]
    )

model.QlossTrDef = Constraint(model.TR_PH, rule=Qloss_tr_rule)



# BT

In [ ]:
# --- Ler BT e PV (mesmo branches_1.xlsx / buses_1.xlsx) ---
df_branches_bt = pd.read_excel(branches_path, sheet_name="BT")
df_buses_bt    = pd.read_excel(buses_path,    sheet_name="BT")
df_pv          = pd.read_excel(buses_path,    sheet_name="PV")  # PV está em buses_1.xlsx

# Normalizar fases em BT e PV
df_branches_bt["phase_list"] = df_branches_bt["phase"].apply(normalize_phases)
df_buses_bt["phase_list"]    = df_buses_bt["phases"].apply(normalize_phases)
df_pv["phase_list"]          = df_pv["phases"].apply(normalize_phases)

# Map name -> N na BT
bus_id_map_bt = dict(zip(df_buses_bt["name"], df_buses_bt["N"]))

buses_bt = df_buses_bt["N"].astype(int).unique().tolist()


In [ ]:
# Mapear ramos BT: l,k (nomes) -> N
df_branches_bt["from_bus"] = df_branches_bt["l"].map(bus_id_map_bt)
df_branches_bt["to_bus"]   = df_branches_bt["k"].map(bus_id_map_bt)
df_branches_bt = df_branches_bt.dropna(subset=["from_bus", "to_bus"])

df_branches_bt["from_bus"] = df_branches_bt["from_bus"].astype(int)
df_branches_bt["to_bus"]   = df_branches_bt["to_bus"].astype(int)
df_branches_bt["m"]        = df_branches_bt["m"].astype(int)

branches_bt = df_branches_bt["m"].unique().tolist()

# Ramos por fase (BT)
branch_phase_bt = []
for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    for ph in row["phase_list"]:
        branch_phase_bt.append((m, ph))
branch_phase_bt = sorted(set(branch_phase_bt))

# Barras por fase (BT)
bus_phase_bt = set()
for (br, ph) in branch_phase_bt:
    fb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "from_bus"].iloc[0])
    tb = int(df_branches_bt.loc[df_branches_bt["m"] == br, "to_bus"].iloc[0])
    bus_phase_bt.add((fb, ph))
    bus_phase_bt.add((tb, ph))
bus_phase_bt = sorted(bus_phase_bt)


In [ ]:
R_bt = {}
X_bt = {}
Imax_bt = {}
from_bus_bt = {}
to_bus_bt = {}
Length_bt_km = {}

for _, row in df_branches_bt.iterrows():
    m = int(float(row["m"]))
    from_bus_bt[m] = int(row["from_bus"])
    to_bus_bt[m]   = int(row["to_bus"])
    Imax_bt[m]     = float(row["Imax"])

    Lkm = float(row["Length_m"]) / 1000.0
    Length_bt_km[m] = Lkm

    for ph in row["phase_list"]:
        # R,X em ohm/km -> total em ohm
        R_bt[(m, ph)] = float(row["R"]) * Lkm
        X_bt[(m, ph)] = float(row["X"]) * Lkm

# Vbase por ramo BT (kV da barra de origem)
Vbase_bt = {}
for m in branches_bt:
    fb = from_bus_bt[m]
    Vbase_bt[m] = float(df_buses_bt.loc[df_buses_bt["N"] == fb, "v_nom_kv"].iloc[0])

# Smax BT (kVA por fase) para limite linear de corrente
Smax_bt = {m: Vbase_bt[m] * Imax_bt[m] for m in branches_bt}


In [ ]:
Pload_bt = {}
Qload_bt = {}

for _, row in df_buses_bt.iterrows():
    b = int(row["N"])
    phs = row["phase_list"]
    if len(phs) == 0:
        continue

    Pd = float(row["P_D"]) if not pd.isna(row["P_D"]) else 0.0
    Qd = float(row["Q_D"]) if not pd.isna(row["Q_D"]) else 0.0

    share = 1.0 / len(phs)

    for ph in phs:
        Pload_bt[(b, ph)] = Pd * share
        Qload_bt[(b, ph)] = Qd * share


In [ ]:
# PV por barra-fase
Ppv_bt = {}     # param (injeção ativa)
Qpv_max = {}    # param (limite reativo)

# Inicializa com zero para todas as barras/fases (se não houver PV)
for (b, ph) in bus_phase_bt:
    Ppv_bt[(b, ph)] = 0.0
    Qpv_max[(b, ph)] = 0.0

for _, row in df_pv.iterrows():
    bus_name = row["Bus"]
    if bus_name not in bus_id_map_bt:
        continue
    b = int(bus_id_map_bt[bus_name])

    # para cada fase, lê colunas p_pv_x e q_pv_x
    for ph in row["phase_list"]:
        pcol = f"p_pv_{ph}"
        qcol = f"q_pv_{ph}"
        Ppv_bt[(b, ph)] = float(row[pcol]) if pcol in row and not pd.isna(row[pcol]) else 0.0
        Qpv_max[(b, ph)] = float(row[qcol]) if qcol in row and not pd.isna(row[qcol]) else 0.0


In [ ]:
# Mapear lv_bus (nome) -> N BT
df_trafos["lv_bus_num"] = df_trafos["lv_bus"].map(bus_id_map_bt)
df_trafos = df_trafos.dropna(subset=["lv_bus_num"])
df_trafos["lv_bus_num"] = df_trafos["lv_bus_num"].astype(int)

trafo_lv_bus = {row["trafo_id"]: int(row["lv_bus_num"]) for _, row in df_trafos.iterrows()}


In [ ]:
# Conjuntos BT
model.BUS_BT = Set(initialize=buses_bt)
model.BUS_PH_BT = Set(dimen=2, initialize=bus_phase_bt)

model.BR_BT = Set(initialize=branches_bt)
model.BR_PH_BT = Set(dimen=2, initialize=branch_phase_bt)

# Parâmetros BT
model.R_BT = Param(model.BR_PH_BT, initialize=R_bt)
model.X_BT = Param(model.BR_PH_BT, initialize=X_bt)

model.from_bus_bt = Param(model.BR_BT, initialize=from_bus_bt)
model.to_bus_bt   = Param(model.BR_BT, initialize=to_bus_bt)

model.Pload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Pload_bt.get((b,ph), 0.0), default=0.0)
model.Qload_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qload_bt.get((b,ph), 0.0), default=0.0)

model.Ppv_bt = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Ppv_bt.get((b,ph), 0.0), default=0.0)
model.Qpv_max = Param(model.BUS_BT, model.PH, initialize=lambda m,b,ph: Qpv_max.get((b,ph), 0.0), default=0.0)


In [ ]:
# Variáveis BT
model.P_BT = Var(model.BR_PH_BT)
model.Q_BT = Var(model.BR_PH_BT)
model.v_BT = Var(model.BUS_BT, model.PH, bounds=(0.95**2, 1.05**2))

# Decomposição abs BT
model.Ppos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Pneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qpos_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)
model.Qneg_BT = Var(model.BR_PH_BT, domain=NonNegativeReals)

# PV: q variável por barra-fase (inicial 0), limitada por Qpv_max
def qpv_bounds(m, b, ph):
    qmax = value(m.Qpv_max[b,ph])
    return (-qmax, qmax)

model.Qpv = Var(model.BUS_BT, model.PH, bounds=qpv_bounds, initialize=0.0)

# PF como variável "rastreamento" (init 1). (acoplamento pf↔Q será feito depois via MILP/PWL)
model.pf_pv = Var(model.BUS_BT, model.PH, bounds=(0.0, 1.0), initialize=1.0)


In [ ]:
def vdrop_bt_rule(m, br, ph):
    i = m.from_bus_bt[br]
    j = m.to_bus_bt[br]
    return m.v_BT[j,ph] == m.v_BT[i,ph] - 2*(m.R_BT[br,ph]*m.P_BT[br,ph] + m.X_BT[br,ph]*m.Q_BT[br,ph])

model.VdropBT = Constraint(model.BR_PH_BT, rule=vdrop_bt_rule)


In [ ]:
def pbal_bt_rule(m, bus, ph):

    inflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.P_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    # Soma potência dos trafos cujo lv_bus é esta barra (pode haver mais de um)
    Ptr_here = sum(
        m.Ptr_pos[tid,ph] - m.Ptr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Pload_bt[bus,ph] - m.Ppv_bt[bus,ph] - Ptr_here

model.PBalanceBT = Constraint(model.BUS_PH_BT, rule=pbal_bt_rule)


def qbal_bt_rule(m, bus, ph):

    inflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.to_bus_bt[br]==bus)
    outflow = sum(m.Q_BT[br,ph] for (br,p) in m.BR_PH_BT if p==ph and m.from_bus_bt[br]==bus)

    Qtr_here = sum(
        m.Qtr_pos[tid,ph] - m.Qtr_neg[tid,ph]
        for (tid,p) in m.TR_PH
        if p==ph and trafo_lv_bus.get(tid, None) == bus
    )

    return inflow - outflow == m.Qload_bt[bus,ph] - m.Qpv[bus,ph] - Qtr_here

model.QBalanceBT = Constraint(model.BUS_PH_BT, rule=qbal_bt_rule)


In [ ]:
def Psplit_bt_rule(m, br, ph):
    return m.P_BT[br,ph] == m.Ppos_BT[br,ph] - m.Pneg_BT[br,ph]
def Qsplit_bt_rule(m, br, ph):
    return m.Q_BT[br,ph] == m.Qpos_BT[br,ph] - m.Qneg_BT[br,ph]

model.PsplitBT = Constraint(model.BR_PH_BT, rule=Psplit_bt_rule)
model.QsplitBT = Constraint(model.BR_PH_BT, rule=Qsplit_bt_rule)

Vmin_bt = 0.95

def current_limit_bt_rule(m, br, ph):
    return (m.Ppos_BT[br,ph] + m.Pneg_BT[br,ph] + m.Qpos_BT[br,ph] + m.Qneg_BT[br,ph]) <= (Smax_bt[br] * Vmin_bt)

model.CurrentLimitBT = Constraint(model.BR_PH_BT, rule=current_limit_bt_rule)


In [ ]:
bt_slacks = df_buses_bt[df_buses_bt["tb"] == 3]["N"].astype(int).tolist()
if len(bt_slacks) > 0:
    bt_slack = bt_slacks[0]
    for ph in PHASES:
        model.v_BT[bt_slack, ph].fix(1.0)


# Regulador

In [ ]:
# Mapeamento explícito regulador → fase
reg_phase_map = {
    "Reg1": 1,
    "Reg2": 2,
    "Reg3": 3,
}

reg_phase = [(rid, ph) for rid, ph in reg_phase_map.items()]

model.REG_PH = Set(dimen=2, initialize=reg_phase)



This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
reg_mv_bus = {row["trafo_id"]: int(bus_id_map[row["mv_bus"]]) for _, row in df_reg.iterrows()}
reg_lv_bus = {row["trafo_id"]: int(bus_id_map[row["lv_bus"]]) for _, row in df_reg.iterrows()}

vreg = {}
band = {}
step = {}
tap_min = {}
tap_max = {}

NTAPS = 16  # <<< DEFINIÇÃO CORRETA (engenharia)

for _, row in df_reg.iterrows():
    rid = row["trafo_id"]
    if rid not in reg_phase_map:
        continue

    ph = reg_phase_map[rid]

    vreg[(rid,ph)] = row["vreg_V"] / (row["kv_mv"] * 1000)
    band[(rid,ph)] = row["band_V"] / (row["kv_mv"] * 1000)

    # Step (% → pu)
    step[(rid,ph)] = float(row["Step"]) / 100.0

    tap_min[(rid,ph)] = -NTAPS
    tap_max[(rid,ph)] =  NTAPS


In [ ]:
model.tap = Var(model.REG_PH, model.T, domain=Integers)


In [ ]:
def tap_bounds(m, rid, ph, t):
    return (tap_min[(rid,ph)], tap_max[(rid,ph)])

model.tap = Var(model.REG_PH, model.T, domain=Integers, bounds=tap_bounds)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
# NÃO aplicar regulador se j == mv_bus do trafo
def reg_voltage(mm, rid, ph, t):
    i = reg_mv_bus[rid]
    j = reg_lv_bus[rid]
    if j in trafo_mv_bus.values():
        return Constraint.Skip
    return mm.v_MT[j,ph,t] == mm.v_MT[i,ph,t] + 2*step[(rid,ph)]*mm.tap[rid,ph,t]


model.RegVoltage = Constraint(model.REG_PH, model.T, rule=reg_voltage_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
model.over_v = Var(model.REG_PH, model.T, domain=Binary)
model.under_v = Var(model.REG_PH, model.T, domain=Binary)


In [ ]:
M = 10.0

def over_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return m.v[j,ph] - (vreg[(rid,ph)] + band[(rid,ph)]/2) <= M * m.over_v[rid,ph,t]

def under_voltage_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return (vreg[(rid,ph)] - band[(rid,ph)]/2) - m.v[j,ph] <= M * m.under_v[rid,ph,t]

model.OverV = Constraint(model.REG_PH, model.T, rule=over_voltage_rule)
model.UnderV = Constraint(model.REG_PH, model.T, rule=under_voltage_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
t0 = T_list[0]
for (rid,ph) in m.REG_PH:
    m.tap[rid,ph,t0].fix(0)
    m.tap_up[rid,ph,t0].fix(0)
    m.tap_down[rid,ph,t0].fix(0)



def tap_evolution_rule(m, rid, ph, t):
    if t == 0:
        return Constraint.Skip
    return (
        m.tap[rid,ph,t]
        == m.tap[rid,ph,t-1]
        + m.tap_up[rid,ph,t]
        - m.tap_down[rid,ph,t]
    )

model.TapEvolution = Constraint(model.REG_PH, model.T, rule=tap_evolution_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
def tap_up_rule(m, rid, ph, t):
    return m.tap_up[rid,ph,t] <= m.under_v[rid,ph,t]

def tap_down_rule(m, rid, ph, t):
    return m.tap_down[rid,ph,t] <= m.over_v[rid,ph,t]

model.TapUpRule = Constraint(model.REG_PH, model.T, rule=tap_up_rule)
model.TapDownRule = Constraint(model.REG_PH, model.T, rule=tap_down_rule)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
model.obj = Objective(
    expr=sum(
        model.tap_up[rid,ph,t] + model.tap_down[rid,ph,t]
        for (rid,ph) in model.REG_PH
        for t in model.T
    ),
    sense=minimize
)


This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [ ]:
solver = SolverFactory("highs")
result = solver.solve(model, tee=True)

print(result.solver.termination_condition)


Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 51855 rows; 43221 cols; 129633 nonzeros; 43215 integer variables (34572 binary)
Coefficient ranges:
  Matrix  [1e-04, 1e+01]
  Cost    [1e+00, 1e+00]
  Bound   [9e-01, 2e+01]
  RHS     [5e-02, 5e-02]
Presolving model
25923 rows, 25929 cols, 69129 nonzeros  0s
0 rows, 8643 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-51855); columns 0(-43221); nonzeros 0(-129633) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  D

# V2

In [219]:
!pip -q install pyomo pandas openpyxl highspy



In [220]:
# base_path = "/content/ACOPF"

branches_path = f"{base_path}/branches_1.xlsx"
buses_path    = f"{base_path}/buses_1.xlsx"

# ----------------------------
# Helpers: normalize phases
# ----------------------------
def normalize_phases(value):
    if pd.isna(value):
        return []
    s = str(value).strip()
    s = s.replace(".", ",")
    parts = [p.strip() for p in s.split(",") if p.strip() != ""]
    phases = []
    for p in parts:
        try:
            ph = int(float(p))
            if ph in [1,2,3]:
                phases.append(ph)
        except ValueError:
            pass
    return sorted(set(phases))

PHASES = [1,2,3]


In [ ]:
# ============================================================
# CONSTANTES FÍSICAS E DE CONTROLE
# ============================================================

delta_v = 0.00625      # step de tap (0.625%) – validado OpenDSS
v_min   = 0.93         # limite inferior de tensão (pu)
v_max   = 1.05         # limite superior de tensão (pu)

pf_min  = 0.90
k_pf    = math.tan(math.acos(pf_min))

# faixa de tap observada no OpenDSS
tap_ratio_min = 1.000
tap_ratio_max = 1.031

# conversão para número de taps
NTAPS = int(round((tap_ratio_max - 1.0) / delta_v))  # ≈ 5


In [221]:
# ----------------------------
# TIME SETUP
# ----------------------------
# Para testes: use range(0, 96) ou range(0, 288)
# T = list(range(0, 2881))  # 0..2880
SOLAR_START = 196
SOLAR_END   = 387

T = list(range(SOLAR_START, SOLAR_END + 1))


In [222]:
# ============================================================
#  LOAD DATA: MT  (COM REMOÇÃO DO RAMO 650->RG60 ANTES DOS SETS)
# ============================================================
df_branches_mt = pd.read_excel(branches_path, sheet_name="MT")
df_trafos      = pd.read_excel(branches_path, sheet_name="Trafos")
df_buses_mt    = pd.read_excel(buses_path,    sheet_name="MT")

# normalize
df_branches_mt["phase_list"] = df_branches_mt["phase"].apply(normalize_phases)
df_trafos["phase_list"]      = df_trafos["phases"].apply(normalize_phases)

# remove SourceBus (AT)
df_buses_mt = df_buses_mt[~df_buses_mt["name"].astype(str).str.contains("Source", case=False)]
bus_id_map_mt = dict(zip(df_buses_mt["name"], df_buses_mt["N"]))

# ---- CORREÇÃO 1: REMOVER O RAMO DO REGULADOR ANTES DE CONSTRUIR SETS/ARAMs ----
# (no OpenDSS Reg1/2/3 são transformers entre 650 e RG60. Logo o ramo MT 650->RG60 NÃO EXISTE como linha.)
df_branches_mt = df_branches_mt[
    ~((df_branches_mt["l"].astype(str) == "650") & (df_branches_mt["k"].astype(str) == "RG60"))
].copy()

buses_mt = df_buses_mt["N"].astype(int).unique().tolist()

# map branch endpoints to numeric IDs
df_branches_mt["from_bus"] = df_branches_mt["l"].map(bus_id_map_mt)
df_branches_mt["to_bus"]   = df_branches_mt["k"].map(bus_id_map_mt)
df_branches_mt = df_branches_mt.dropna(subset=["from_bus","to_bus"]).copy()
df_branches_mt["from_bus"] = df_branches_mt["from_bus"].astype(int)
df_branches_mt["to_bus"]   = df_branches_mt["to_bus"].astype(int)
df_branches_mt["m"]        = df_branches_mt["m"].astype(int)

branches_mt = df_branches_mt["m"].unique().tolist()

# branch-phase set (MT)
BR_PH_MT = sorted({(int(row["m"]), ph) for _,row in df_branches_mt.iterrows() for ph in row["phase_list"]})

# bus-phase set (MT)
BUS_PH_MT = set()
from_bus_mt = {}
to_bus_mt   = {}
Imax_mt     = {}
for _, row in df_branches_mt.iterrows():
    m = int(row["m"])
    from_bus_mt[m] = int(row["from_bus"])
    to_bus_mt[m]   = int(row["to_bus"])
    Imax_mt[m]     = float(row["Imax"])

for (br,ph) in BR_PH_MT:
    BUS_PH_MT.add((from_bus_mt[br], ph))
    BUS_PH_MT.add((to_bus_mt[br], ph))
BUS_PH_MT = sorted(BUS_PH_MT)

slack_bus_mt = int(df_buses_mt[df_buses_mt["tb"] == 3]["N"].iloc[0])


In [224]:
# ============================================================
#  MT R/X (assumindo R e X já são totais por ramo, NÃO ohm/km)
# ============================================================
R_mt = {}
X_mt = {}

for _, row in df_branches_mt.iterrows():
    m_id = int(row["m"])
    for ph in row["phase_list"]:
        R_mt[(m_id, ph)] = float(row["R"])
        X_mt[(m_id, ph)] = float(row["X"])

In [225]:
# ============================================================
#  TRANSFORMERS (MT<->BT) coupling sets + maps
# ============================================================
df_trafos["mv_bus_num"] = df_trafos["mv_bus"].map(bus_id_map_mt)
df_trafos["lv_bus_num"] = df_trafos["lv_bus"].map(bus_id_map_bt)
df_trafos = df_trafos.dropna(subset=["mv_bus_num","lv_bus_num"]).copy()
df_trafos["mv_bus_num"] = df_trafos["mv_bus_num"].astype(int)
df_trafos["lv_bus_num"] = df_trafos["lv_bus_num"].astype(int)

TR_PH = sorted({(row["trafo_id"], ph) for _,row in df_trafos.iterrows() for ph in row["phase_list"]})
trafo_mv_bus = {row["trafo_id"]: int(row["mv_bus_num"]) for _,row in df_trafos.iterrows()}
trafo_lv_bus = {row["trafo_id"]: int(row["lv_bus_num"]) for _,row in df_trafos.iterrows()}

# transformer loss coefficients (linear envelope)
alpha_tr = {}
beta_tr  = {}
for _, row in df_trafos.iterrows():
    tid = row["trafo_id"]
    r_per = float(row["r_per"]) / 100.0 if not pd.isna(row["r_per"]) else 0.0
    xhl   = float(row["XHL"])   / 100.0 if not pd.isna(row["XHL"])   else 0.0
    for ph in row["phase_list"]:
        alpha_tr[(tid,ph)] = r_per
        beta_tr[(tid,ph)]  = xhl

In [226]:
# ============================================================
#  REGULATORS (SVR) on MT (sheet Reg)
# ============================================================
df_reg = pd.read_excel(branches_path, sheet_name="Reg")

reg_phase_map = {"Reg1":1, "Reg2":2, "Reg3":3}
REG_PH = sorted([(rid,ph) for rid,ph in reg_phase_map.items()])

reg_mv_bus = {row["trafo_id"]: int(bus_id_map_mt[row["mv_bus"]]) for _,row in df_reg.iterrows() if row["trafo_id"] in reg_phase_map}
reg_lv_bus = {row["trafo_id"]: int(bus_id_map_mt[row["lv_bus"]]) for _,row in df_reg.iterrows() if row["trafo_id"] in reg_phase_map}

vreg = {}
band = {}
step = {}
NTAPS = 16
tap_min = {(rid,ph):-NTAPS for (rid,ph) in REG_PH}
tap_max = {(rid,ph): NTAPS for (rid,ph) in REG_PH}

# ----- CORREÇÃO 2: potência nominal do regulador por fase (kVA/fase) -----
Sreg_max = {}

for _, row in df_reg.iterrows():
    rid = row["trafo_id"]
    if rid not in reg_phase_map:
        continue
    ph = reg_phase_map[rid]
    kv = float(row["kv_mv"])
    vreg[(rid,ph)] = float(row["vreg_V"]) / (kv*1000.0)
    band[(rid,ph)] = float(row["band_V"]) / (kv*1000.0)
    step[(rid,ph)] = float(row["Step"]) / 100.0

    # sheet Reg: 'kva' é total do regulador monofásico (no OpenDSS: 1666 por fase)
    kva = float(row["kva"]) if not pd.isna(row["kva"]) else 0.0
    Sreg_max[(rid,ph)] = kva




In [227]:
# ============================================================
#  MT apparent power limit per branch (kVA por fase)
# ============================================================
Vmin_pu = 0.95  # <-- isso também estava faltando
Smax_mt = {}

for m_id in branches_mt:
    fb = from_bus_mt[m_id]
    Vbase = float(df_buses_mt.loc[df_buses_mt["N"] == fb, "v_nom_kv"].iloc[0])
    Smax_mt[m_id] = Vbase * Imax_mt[m_id]  # kVA por fase


In [228]:
# ============================================================
#  PYOMO MODEL
# ============================================================
m = ConcreteModel()

# Sets
m.T   = Set(initialize=T, ordered=True)
m.PH  = Set(initialize=PHASES)

m.BUS_MT    = Set(initialize=buses_mt)
m.BUS_PH_MT = Set(dimen=2, initialize=BUS_PH_MT)
m.BR_MT     = Set(initialize=branches_mt)
m.BR_PH_MT  = Set(dimen=2, initialize=BR_PH_MT)

m.BUS_BT    = Set(initialize=buses_bt)
m.BUS_PH_BT = Set(dimen=2, initialize=BUS_PH_BT)
m.BR_BT     = Set(initialize=branches_bt)
m.BR_PH_BT  = Set(dimen=2, initialize=BR_PH_BT)

m.TR_PH     = Set(dimen=2, initialize=TR_PH)
m.REG_PH    = Set(dimen=2, initialize=REG_PH)

# ----- CORREÇÃO 3: subestação como fonte (equivalente a SourceBus removido) -----
m.Psub = Var(m.PH, m.T, bounds=(0, None))   # fonte ativa (kW/kVA coerente com seu modelo)
m.Qsub = Var(m.PH, m.T)                    # fonte reativa (livre)


In [229]:
# Params MT
m.R_MT = Param(m.BR_PH_MT, initialize=R_mt)
m.X_MT = Param(m.BR_PH_MT, initialize=X_mt)
m.from_mt = Param(m.BR_MT, initialize=from_bus_mt)
m.to_mt   = Param(m.BR_MT, initialize=to_bus_mt)

# Params BT
m.R_BT = Param(m.BR_PH_BT, initialize=R_bt)
m.X_BT = Param(m.BR_PH_BT, initialize=X_bt)
m.from_bt = Param(m.BR_BT, initialize=from_bus_bt)
m.to_bt   = Param(m.BR_BT, initialize=to_bus_bt)

m.Pload_bt = Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Pload_bt.get((b,ph),0.0), default=0.0)
m.Qload_bt = Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Qload_bt.get((b,ph),0.0), default=0.0)

m.Ppv_max = Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Ppv_max.get((b,ph),0.0), default=0.0)
m.Qpv_max = Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Qpv_max.get((b,ph),0.0), default=0.0)
m.Spv_max = Param(m.BUS_BT, m.PH, initialize=lambda mm,b,ph: Spv_max.get((b,ph),0.0), default=0.0)

In [230]:
# Parametros Regulador
# Valores estáveis em pu (evita infeasibilidade numérica)
R_reg = {(rid,ph): 0.05 for (rid,ph) in m.REG_PH}
X_reg = {(rid,ph): 0.10 for (rid,ph) in m.REG_PH}



In [231]:


# ----------------------------
# Variables MT (time-indexed)
# ----------------------------
m.P_MT = Var(m.BR_PH_MT, m.T)
m.Q_MT = Var(m.BR_PH_MT, m.T)
m.v_MT = Var(m.BUS_MT, m.PH, m.T, bounds=(0.95**2, 1.05**2))

m.Ppos_MT = Var(m.BR_PH_MT, m.T, domain=NonNegativeReals)
m.Pneg_MT = Var(m.BR_PH_MT, m.T, domain=NonNegativeReals)
m.Qpos_MT = Var(m.BR_PH_MT, m.T, domain=NonNegativeReals)
m.Qneg_MT = Var(m.BR_PH_MT, m.T, domain=NonNegativeReals)

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().
This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [232]:
# Line losses (optional, diagnostic)
m.Ploss_MT = Var(m.BR_PH_MT, m.T, domain=NonNegativeReals)

In [233]:
# ----------------------------
# Trafo coupling variables (shared MT<->BT), time-indexed
# ----------------------------
m.Ptr_pos = Var(m.TR_PH, m.T, domain=NonNegativeReals)
m.Ptr_neg = Var(m.TR_PH, m.T, domain=NonNegativeReals)
m.Qtr_pos = Var(m.TR_PH, m.T, domain=NonNegativeReals)
m.Qtr_neg = Var(m.TR_PH, m.T, domain=NonNegativeReals)

m.Ploss_tr = Var(m.TR_PH, m.T, domain=NonNegativeReals)
m.Qloss_tr = Var(m.TR_PH, m.T, domain=NonNegativeReals)


In [234]:
# ----------------------------
# Variables BT (time-indexed)
# ----------------------------
m.P_BT = Var(m.BR_PH_BT, m.T)
m.Q_BT = Var(m.BR_PH_BT, m.T)
m.v_BT = Var(m.BUS_BT, m.PH, m.T, bounds=(0.95**2, 1.05**2))

m.Ppos_BT = Var(m.BR_PH_BT, m.T, domain=NonNegativeReals)
m.Pneg_BT = Var(m.BR_PH_BT, m.T, domain=NonNegativeReals)
m.Qpos_BT = Var(m.BR_PH_BT, m.T, domain=NonNegativeReals)
m.Qneg_BT = Var(m.BR_PH_BT, m.T, domain=NonNegativeReals)

# PV variables (time-indexed)
m.Ppv = Var(m.BUS_BT, m.PH, m.T, domain=NonNegativeReals, initialize=0.0)
m.Qpv = Var(m.BUS_BT, m.PH, m.T, initialize=0.0)
m.Qpv_pos = Var(m.BUS_BT, m.PH, m.T, domain=NonNegativeReals)
m.Qpv_neg = Var(m.BUS_BT, m.PH, m.T, domain=NonNegativeReals)

# PF tracking (not coupled nonlinearly here)
m.pf_pv = Var(m.BUS_BT, m.PH, m.T, bounds=(0.0,1.0), initialize=1.0)


In [235]:
# Variaveis Regulador

# ============================================================
# REGULADOR DE TENSÃO (SVR)
# ============================================================

# Tap inteiro
m.tap = Var(
    m.REG_PH, m.T,
    domain=Integers,
    bounds=lambda m,rid,ph,t: (-NTAPS, NTAPS)
)

# Lógica de controle
m.tap_up   = Var(m.REG_PH, m.T, domain=Binary)
m.tap_down = Var(m.REG_PH, m.T, domain=Binary)

m.over_v  = Var(m.REG_PH, m.T, domain=Binary)
m.under_v = Var(m.REG_PH, m.T, domain=Binary)

m.Preg = Var(m.REG_PH, m.T)
m.Qreg = Var(m.REG_PH, m.T)

m.Preg_pos = Var(m.REG_PH, m.T, domain=NonNegativeReals)
m.Preg_neg = Var(m.REG_PH, m.T, domain=NonNegativeReals)
m.Qreg_pos = Var(m.REG_PH, m.T, domain=NonNegativeReals)
m.Qreg_neg = Var(m.REG_PH, m.T, domain=NonNegativeReals)



In [236]:
# ----------------------------
# SVR (Regulator) variables (time-indexed)
# ----------------------------
def tap_bounds(mm, rid, ph, t):
    return (tap_min[(rid,ph)], tap_max[(rid,ph)])

m.tap = Var(m.REG_PH, m.T, domain=Integers, bounds=tap_bounds)

m.tap_up   = Var(m.REG_PH, m.T, domain=Binary)
m.tap_down = Var(m.REG_PH, m.T, domain=Binary)

m.over_v  = Var(m.REG_PH, m.T, domain=Binary)
m.under_v = Var(m.REG_PH, m.T, domain=Binary)



In [237]:
# ============================================================
#  CONSTRAINTS: Splits + current limits (MT)
# ============================================================
def Psplit_mt(mm, br, ph, t):
    return mm.P_MT[br,ph,t] == mm.Ppos_MT[br,ph,t] - mm.Pneg_MT[br,ph,t]
def Qsplit_mt(mm, br, ph, t):
    return mm.Q_MT[br,ph,t] == mm.Qpos_MT[br,ph,t] - mm.Qneg_MT[br,ph,t]
m.PsplitMT = Constraint(m.BR_PH_MT, m.T, rule=Psplit_mt)
m.QsplitMT = Constraint(m.BR_PH_MT, m.T, rule=Qsplit_mt)

def current_limit_mt(mm, br, ph, t):
    return (mm.Ppos_MT[br,ph,t] + mm.Pneg_MT[br,ph,t] + mm.Qpos_MT[br,ph,t] + mm.Qneg_MT[br,ph,t]) <= (Smax_mt[br]*Vmin_pu)
m.CurrentLimitMT = Constraint(m.BR_PH_MT, m.T, rule=current_limit_mt)

# Optional line loss envelope (diagnostic)
def ploss_mt_rule(mm, br, ph, t):
    return mm.Ploss_MT[br,ph,t] >= mm.R_MT[br,ph]*(mm.Ppos_MT[br,ph,t] + mm.Pneg_MT[br,ph,t] + mm.Qpos_MT[br,ph,t] + mm.Qneg_MT[br,ph,t])
m.PlossMTDef = Constraint(m.BR_PH_MT, m.T, rule=ploss_mt_rule)


In [238]:
# ============================================================
#  CONSTRAINTS: LinDistFlow voltage drops (MT)
# ============================================================
def vdrop_mt(mm, br, ph, t):
    i = mm.from_mt[br]
    j = mm.to_mt[br]
    return mm.v_MT[j,ph,t] == mm.v_MT[i,ph,t] - 2*(mm.R_MT[br,ph]*mm.P_MT[br,ph,t] + mm.X_MT[br,ph]*mm.Q_MT[br,ph,t])
m.VdropMT = Constraint(m.BR_PH_MT, m.T, rule=vdrop_mt)

# ============================================================
#  CONSTRAINTS: Trafo losses (linear envelope)
# ============================================================
def ploss_tr_rule(mm, tid, ph, t):
    a = alpha_tr.get((tid,ph), 0.0)
    return mm.Ploss_tr[tid,ph,t] >= a*(mm.Ptr_pos[tid,ph,t] + mm.Ptr_neg[tid,ph,t] + mm.Qtr_pos[tid,ph,t] + mm.Qtr_neg[tid,ph,t])
m.PlossTR = Constraint(m.TR_PH, m.T, rule=ploss_tr_rule)

def qloss_tr_rule(mm, tid, ph, t):
    b = beta_tr.get((tid,ph), 0.0)
    return mm.Qloss_tr[tid,ph,t] >= b*(mm.Ptr_pos[tid,ph,t] + mm.Ptr_neg[tid,ph,t] + mm.Qtr_pos[tid,ph,t] + mm.Qtr_neg[tid,ph,t])
m.QlossTR = Constraint(m.TR_PH, m.T, rule=qloss_tr_rule)


In [239]:
def pbal_mt(mm, bus, ph, t):
    inflow  = sum(mm.P_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.to_mt[br]==bus)
    outflow = sum(mm.P_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.from_mt[br]==bus)

    tr_term = sum((mm.Ptr_pos[tid,ph,t] - mm.Ptr_neg[tid,ph,t]) + mm.Ploss_tr[tid,ph,t]
                  for (tid,p) in mm.TR_PH if p==ph and trafo_mv_bus[tid]==bus)

    reg_in  = sum(mm.Preg[rid,ph,t]
                  for (rid,p) in mm.REG_PH
                  if p==ph and reg_lv_bus[rid]==bus)

    reg_out = sum(mm.Preg[rid,ph,t] + mm.Ploss_reg[rid,ph,t]
                  for (rid,p) in mm.REG_PH
                  if p==ph and reg_mv_bus[rid]==bus)

    sub = mm.Psub[ph,t] if bus == slack_bus_mt else 0.0

    return inflow - outflow + reg_in - reg_out + sub == tr_term

m.PBalMT = Constraint(m.BUS_PH_MT, m.T, rule=pbal_mt)


def qbal_mt(mm, bus, ph, t):
    inflow  = sum(mm.Q_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.to_mt[br]==bus)
    outflow = sum(mm.Q_MT[br,p,t] for (br,p) in mm.BR_PH_MT if p==ph and mm.from_mt[br]==bus)

    tr_term = sum((mm.Qtr_pos[tid,ph,t] - mm.Qtr_neg[tid,ph,t]) + mm.Qloss_tr[tid,ph,t]
                  for (tid,p) in mm.TR_PH if p==ph and trafo_mv_bus[tid]==bus)

    reg_in  = sum(mm.Qreg[rid,ph,t]
                  for (rid,p) in mm.REG_PH
                  if p==ph and reg_lv_bus[rid]==bus)

    reg_out = sum(mm.Qreg[rid,ph,t] + mm.Qloss_reg[rid,ph,t]
                  for (rid,p) in mm.REG_PH
                  if p==ph and reg_mv_bus[rid]==bus)

    sub = mm.Qsub[ph,t] if bus == slack_bus_mt else 0.0

    return inflow - outflow + reg_in - reg_out + sub == tr_term

m.QBalMT = Constraint(m.BUS_PH_MT, m.T, rule=qbal_mt)


In [240]:
# ============================================================
#  CONSTRAINTS: PV capability + solar window
# ============================================================
def qpv_split(mm, b, ph, t):
    return mm.Qpv[b,ph,t] == mm.Qpv_pos[b,ph,t] - mm.Qpv_neg[b,ph,t]
m.QpvSplit = Constraint(m.BUS_BT, m.PH, m.T, rule=qpv_split)

def qpv_limit(mm, b, ph, t):
    return mm.Qpv_pos[b,ph,t] + mm.Qpv_neg[b,ph,t] <= mm.Qpv_max[b,ph]
m.QpvLimit = Constraint(m.BUS_BT, m.PH, m.T, rule=qpv_limit)

def pv_capability(mm, b, ph, t):
    return mm.Ppv[b,ph,t] + mm.Qpv_pos[b,ph,t] + mm.Qpv_neg[b,ph,t] <= mm.Spv_max[b,ph]
m.PVCapability = Constraint(m.BUS_BT, m.PH, m.T, rule=pv_capability)

def pv_time_window(mm, b, ph, t):
    if t < SOLAR_START or t > SOLAR_END:
        return mm.Ppv[b,ph,t] == 0
    return mm.Ppv[b,ph,t] <= mm.Ppv_max[b,ph]
m.PVTimeWindow = Constraint(m.BUS_BT, m.PH, m.T, rule=pv_time_window)

def qpv_off(mm, b, ph, t):
    if t < SOLAR_START or t > SOLAR_END:
        return mm.Qpv[b,ph,t] == 0
    return Constraint.Skip
m.QPVOff = Constraint(m.BUS_BT, m.PH, m.T, rule=qpv_off)

FP_MIN = 0.92
K_FP = math.tan(math.acos(FP_MIN))

def fp_limit(m, b, ph, t):
    return (
        m.Qpv_pos[b,ph,t] + m.Qpv_neg[b,ph,t]
        <= k_pf * m.Ppv[b,ph,t]
    )

m.PV_FPLimit = Constraint(m.BUS_BT, m.PH, m.T, rule=fp_limit)


In [241]:
# ============================================================
#  CONSTRAINTS: Splits + current limits (BT)
# ============================================================
def Psplit_bt(mm, br, ph, t):
    return mm.P_BT[br,ph,t] == mm.Ppos_BT[br,ph,t] - mm.Pneg_BT[br,ph,t]
def Qsplit_bt(mm, br, ph, t):
    return mm.Q_BT[br,ph,t] == mm.Qpos_BT[br,ph,t] - mm.Qneg_BT[br,ph,t]
m.PsplitBT = Constraint(m.BR_PH_BT, m.T, rule=Psplit_bt)
m.QsplitBT = Constraint(m.BR_PH_BT, m.T, rule=Qsplit_bt)

def current_limit_bt(mm, br, ph, t):
    return (mm.Ppos_BT[br,ph,t] + mm.Pneg_BT[br,ph,t] + mm.Qpos_BT[br,ph,t] + mm.Qneg_BT[br,ph,t]) <= (Smax_bt[br]*Vmin_pu)
m.CurrentLimitBT = Constraint(m.BR_PH_BT, m.T, rule=current_limit_bt)

In [242]:
# ============================================================
#  CONSTRAINTS: LinDistFlow voltage drops (BT)
# ============================================================
def vdrop_bt(mm, br, ph, t):
    i = mm.from_bt[br]
    j = mm.to_bt[br]
    return mm.v_BT[j,ph,t] == mm.v_BT[i,ph,t] - 2*(mm.R_BT[br,ph]*mm.P_BT[br,ph,t] + mm.X_BT[br,ph]*mm.Q_BT[br,ph,t])
m.VdropBT = Constraint(m.BR_PH_BT, m.T, rule=vdrop_bt)


In [243]:
# ============================================================
#  CONSTRAINTS: BT nodal balances with trafo coupling + PV
# ============================================================
def pbal_bt(mm, bus, ph, t):
    inflow  = sum(mm.P_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.to_bt[br]==bus)
    outflow = sum(mm.P_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.from_bt[br]==bus)

    tr_inj = sum((mm.Ptr_pos[tid,ph,t] - mm.Ptr_neg[tid,ph,t])
                 for (tid,p) in mm.TR_PH if p==ph and trafo_lv_bus[tid]==bus)

    return inflow - outflow == mm.Pload_bt[bus,ph] - mm.Ppv[bus,ph,t] - tr_inj
m.PBalBT = Constraint(m.BUS_PH_BT, m.T, rule=pbal_bt)

def qbal_bt(mm, bus, ph, t):
    inflow  = sum(mm.Q_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.to_bt[br]==bus)
    outflow = sum(mm.Q_BT[br,p,t] for (br,p) in mm.BR_PH_BT if p==ph and mm.from_bt[br]==bus)

    tr_inj = sum((mm.Qtr_pos[tid,ph,t] - mm.Qtr_neg[tid,ph,t])
                 for (tid,p) in mm.TR_PH if p==ph and trafo_lv_bus[tid]==bus)

    return inflow - outflow == mm.Qload_bt[bus,ph] - mm.Qpv[bus,ph,t] - tr_inj
m.QBalBT = Constraint(m.BUS_PH_BT, m.T, rule=qbal_bt)

In [244]:
# ============================================================
#  CONSTRAINTS: Trafo voltage coupling (pu)
#  In per-unit (each side on its own base), ideal trafo => Vpu same.
#  So enforce v_BT(lv_bus) == v_MT(mv_bus).
#  If you want ratio explicitly: v_BT == (kv_lv/kv_mv)^2 * v_MT.
# ============================================================
def trafo_v_couple(mm, tid, ph, t):
    mvb = trafo_mv_bus[tid]
    lvb = trafo_lv_bus[tid]
    return mm.v_BT[lvb,ph,t] == mm.v_MT[mvb,ph,t]
m.TrafoVCouple = Constraint(m.TR_PH, m.T, rule=trafo_v_couple)

In [245]:
# ============================================================
#  SVR constraints (Regulator in MT)
# ============================================================

def reg_psplit(m, rid, ph, t):
    return m.Preg[rid,ph,t] == m.Preg_pos[rid,ph,t] - m.Preg_neg[rid,ph,t]

def reg_qsplit(m, rid, ph, t):
    return m.Qreg[rid,ph,t] == m.Qreg_pos[rid,ph,t] - m.Qreg_neg[rid,ph,t]

m.RegPSplit = Constraint(m.REG_PH, m.T, rule=reg_psplit)
m.RegQSplit = Constraint(m.REG_PH, m.T, rule=reg_qsplit)


# Perdas no Regulador
def reg_ploss(mm, rid, ph, t):
    return mm.Ploss_reg[rid,ph,t] >= R_reg[(rid,ph)] * (
        mm.Preg_pos[rid,ph,t] + mm.Preg_neg[rid,ph,t] +
        mm.Qreg_pos[rid,ph,t] + mm.Qreg_neg[rid,ph,t]
    )

m.RegLoss = Constraint(m.REG_PH, m.T, rule=reg_ploss)


# ----- CORREÇÃO 4: limite de potência no regulador (kVA por fase) -----
def reg_power_limit(m, rid, ph, t):
    return (
        m.Preg_pos[rid,ph,t] + m.Preg_neg[rid,ph,t] +
        m.Qreg_pos[rid,ph,t] + m.Qreg_neg[rid,ph,t]
    ) <= Sreg_max[(rid,ph)]

m.RegPowerLimit = Constraint(m.REG_PH, m.T, rule=reg_power_limit)



# NÃO aplicar regulador se j == mv_bus do trafo
def reg_voltage(m, rid, ph, t):
    i = reg_mv_bus[rid]
    j = reg_lv_bus[rid]

    tap_ratio = 1.0 + delta_v * m.tap[rid,ph,t]

    return m.v_MT[j,ph,t] == \
           tap_ratio**2 * m.v_MT[i,ph,t] \
           - 2 * (
               R_reg[(rid,ph)] * m.Preg[rid,ph,t]
             + X_reg[(rid,ph)] * m.Qreg[rid,ph,t]
           )

m.RegVoltage = Constraint(m.REG_PH, m.T, rule=reg_voltage)



In [246]:
# Big-M logic for violation
Mbig = 100.0

def over_v_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return m.v_MT[j,ph,t] - v_max <= Mbig * m.over_v[rid,ph,t]

def under_v_rule(m, rid, ph, t):
    j = reg_lv_bus[rid]
    return v_min - m.v_MT[j,ph,t] <= Mbig * m.under_v[rid,ph,t]

m.OverV  = Constraint(m.REG_PH, m.T, rule=over_v_rule)
m.UnderV = Constraint(m.REG_PH, m.T, rule=under_v_rule)


In [247]:
T_list = sorted(list(m.T.data()))
prev_t = {T_list[i]: T_list[i-1] for i in range(1, len(T_list))}
t0 = T_list[0]

# condição inicial
for (rid,ph) in m.REG_PH:
    m.tap[rid,ph,t0].fix(0)
    m.tap_up[rid,ph,t0].fix(0)
    m.tap_down[rid,ph,t0].fix(0)

def tap_evolution(m, rid, ph, t):
    if t == t0:
        return Constraint.Skip
    return (
        m.tap[rid,ph,t]
        == m.tap[rid,ph,prev_t[t]]
        + m.tap_up[rid,ph,t]
        - m.tap_down[rid,ph,t]
    )

m.TapEvolution = Constraint(m.REG_PH, m.T, rule=tap_evolution)


def tap_up_rule(m, rid, ph, t):
    return m.tap_up[rid,ph,t] <= m.under_v[rid,ph,t]

def tap_down_rule(m, rid, ph, t):
    return m.tap_down[rid,ph,t] <= m.over_v[rid,ph,t]

m.TapUpRule   = Constraint(m.REG_PH, m.T, rule=tap_up_rule)
m.TapDownRule = Constraint(m.REG_PH, m.T, rule=tap_down_rule)



In [248]:
# ============================================================
#  Slack fixing (MT) for all t
# ============================================================
for ph in PHASES:
    for t in T:
        m.v_MT[slack_bus_mt, ph, t].fix(1.0)

# # Optional BT slack (only if you really want; with trafo voltage coupling, not necessary)
# if bt_slack is not None:
#     for ph in PHASES:
#         for t in T:
#             m.v_BT[bt_slack, ph, t].fix(1.0)


In [265]:

# ============================================================
#  Objective: minimize tap operations
# ============================================================
# objetivo
m.obj = Objective(
    expr=
        sum(m.tap_up[rid,ph,t] + m.tap_down[rid,ph,t]
            for (rid,ph) in m.REG_PH for t in m.T)
      + 1e-3 * sum(m.Ploss_MT[br,ph,t]
            for (br,ph) in m.BR_PH_MT for t in m.T),
    sense=minimize
)



This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


In [266]:
# ============================================================
#  Solve (HiGHS MILP)
# ============================================================
solver = SolverFactory("highs")
res = solver.solve(m, tee=True)
print(res.solver.termination_condition)

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms


IncompatibleModelError: Highs interface does not support expressions of degree None